In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
dfs = pd.DataFrame({})

for i in range(4,10):
    df = pd.read_csv('./csv/sms-call-internet-mi-2013-11-0{}.csv'.format(i), parse_dates=['time'])
    dfs = dfs.append(df)
df = pd.read_csv('./csv/sms-call-internet-mi-2013-11-10.csv', parse_dates=['time'])
dfs = dfs.append(df)
dfs = dfs.fillna(0)

In [3]:
#Group by hours and aggregate values according to each cell/hour
dfgr = dfs[['cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['time', 'cellid'], as_index=False).sum()
dfgr['minute'] = dfgr.time.dt.hour*60+24*60*(dfgr.time.dt.day-4)+dfgr.time.dt.minute
dfgg = dfgr[['minute', 'cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['minute', 'cellid'], as_index=False).sum()
#dfgg = dfgg.set_index(['hour']).sort_index()
print(dfgg)

#Group by hours and aggregate values according to the hour
#dfgr = dfs[['cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['time', 'cellid'], as_index=False).sum()
#dfgr['hour'] = dfgr.time.dt.hour+24*(dfgr.time.dt.day-1)
#dfgg = dfgr[['hour', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['hour'], as_index=False).sum()
#dfgg = dfgg.set_index(['hour']).sort_index()
#print(dfgg)

          minute  cellid     smsin    smsout    callin   callout   internet
0              0       1  0.134176  0.030875  0.026137  0.055225   9.260190
1              0       2  0.136613  0.029144  0.027356  0.055578   9.282384
2              0       3  0.139208  0.027300  0.028653  0.055954   9.306009
3              0       4  0.127117  0.035890  0.022608  0.054203   9.195904
4              0       5  0.114418  0.033496  0.021727  0.050303   8.312392
...          ...     ...       ...       ...       ...       ...        ...
10079846   10070    9996  0.292904  1.047605  0.178533  0.453114  38.491634
10079847   10070    9997  0.257984  1.034799  0.171990  0.533672  38.994166
10079848   10070    9998  0.257984  1.034919  0.171990  0.520073  38.959929
10079849   10070    9999  0.346751  0.725508  0.171990  0.237047  27.247603
10079850   10070   10000  0.466230  0.439523  0.171990  0.090769  19.402621

[10079851 rows x 7 columns]


In [4]:
l=[] #choosen cells on 10x10 grid
for i in range(4273,5273,100):
    for j in range(i,i+10):
        l.append(j)

In [5]:
df10 = pd.DataFrame({})
for el in dfgg.iterrows():
    if(l.count(int(el[1].cellid))==1):
        tmp = pd.DataFrame([[(el[1].minute), int(el[1].cellid), el[1].smsin, el[1].smsout, el[1].callin, el[1].callout, el[1].internet]], columns=['minute', 'cellid', 'smsin','smsout', 'callin','callout', 'internet'])
        df10=df10.append(tmp)

In [6]:
df10 = df10.set_index(['minute']).sort_index()
print(df10)

         cellid     smsin    smsout    callin   callout   internet
minute                                                            
0.0        4273  3.710662  0.449703  0.222218  0.807770  48.354397
0.0        4274  1.247336  0.092817  1.093930  0.505056  34.106327
0.0        4275  2.229053  0.896764  1.203314  0.636555  42.775622
0.0        4276  2.186545  3.002982  0.297875  0.337424  74.426438
0.0        4277  1.727746  3.455491  0.287958  0.287958  81.982781
...         ...       ...       ...       ...       ...        ...
10070.0    5178  1.159530  0.486642  0.077306  0.000000  50.853361
10070.0    5179  1.564403  0.380081  0.071380  0.069155  53.426579
10070.0    5180  1.555974  0.417061  0.232713  0.208530  55.051547
10070.0    5181  1.481122  0.334195  0.213672  0.167098  48.976719
10070.0    5182  0.257947  0.024027  0.142400  0.012013  20.252807

[100800 rows x 6 columns]


In [7]:
cells = df10.cellid
scaler=MinMaxScaler()
scaled = pd.DataFrame(scaler.fit_transform(df10), columns=df10.columns, index=df10.index)

In [8]:
dfgg=scaled
dfgg['cellid']=cells
print(dfgg)

         cellid     smsin    smsout    callin   callout  internet
minute                                                           
0.0        4273  0.005099  0.000756  0.000998  0.003260  0.020315
0.0        4274  0.001714  0.000156  0.004914  0.002038  0.014252
0.0        4275  0.003063  0.001508  0.005405  0.002569  0.017941
0.0        4276  0.003005  0.005051  0.001338  0.001362  0.031410
0.0        4277  0.002374  0.005813  0.001294  0.001162  0.034625
...         ...       ...       ...       ...       ...       ...
10070.0    5178  0.001594  0.000819  0.000347  0.000000  0.021378
10070.0    5179  0.002150  0.000639  0.000321  0.000279  0.022473
10070.0    5180  0.002138  0.000702  0.001045  0.000842  0.023165
10070.0    5181  0.002035  0.000562  0.000960  0.000674  0.020580
10070.0    5182  0.000354  0.000040  0.000640  0.000048  0.008357

[100800 rows x 6 columns]


In [9]:
dfsi = [] #data from df without indexing
dfsi2 = pd.DataFrame({}) #new df to save
dfso = []
dfso2 = pd.DataFrame({})
dfci = []
dfci2 = pd.DataFrame({})
dfco = []
dfco2 = pd.DataFrame({})
dfin = []
dfin2 = pd.DataFrame({})

dfsi.append(dfgg.loc[0].cellid)
dfso.append(dfgg.loc[0].cellid)
dfci.append(dfgg.loc[0].cellid)
dfco.append(dfgg.loc[0].cellid)
dfin.append(dfgg.loc[0].cellid)

dfsi2["cellid"]=dfsi[0].values
dfso2["cellid"]=dfsi[0].values
dfci2["cellid"]=dfsi[0].values
dfco2["cellid"]=dfsi[0].values
dfin2["cellid"]=dfsi[0].values
j=0
for i in (np.unique(dfgr.minute)): #dfgr.hour.max()+1
    
    #row[0] - hour, row[1]: df cols
    dfsi.append(dfgg.loc[i].smsin)
    dfsi2["smsin"+str(i)]=dfsi[j+1].values
    
    dfso.append(dfgg.loc[i].smsout)
    dfso2["smsout"+str(i)]=dfso[j+1].values
    
    dfci.append(dfgg.loc[i].callin)
    dfci2["callin"+str(i)]=dfci[j+1].values
    
    dfco.append(dfgg.loc[i].callout)
    dfco2["callout"+str(i)]=dfco[j+1].values
    
    dfin.append(dfgg.loc[i].internet)
    dfin2["internet"+str(i)]=dfin[j+1].values
    j+=1


In [10]:
dfsi2.to_csv("./classif/10x10/classif10x10weekminute_smsin.csv")
dfsi2.head()

,cellid,smsin0,smsin10,smsin20,smsin30,smsin40,smsin50,smsin60,smsin70,smsin80,...,smsin9980,smsin9990,smsin10000,smsin10010,smsin10020,smsin10030,smsin10040,smsin10050,smsin10060,smsin10070
0,4273,0.005099,0.002071,0.000370,0.000070,0.003484,0.000188,0.000634,0.000682,0.000471,...,0.001830,0.001895,0.001841,0.002401,0.002343,0.000829,0.000988,0.000506,0.001137,0.000647
1,4274,0.001714,0.001073,0.000669,0.000037,0.001204,0.000624,0.000719,0.002699,0.000341,...,0.000201,0.000235,0.000412,0.000883,0.000582,0.000086,0.000440,0.000088,0.000335,0.000417
2,4275,0.003063,0.001575,0.001210,0.000336,0.001079,0.001007,0.000729,0.002813,0.000314,...,0.001844,0.001775,0.000867,0.002087,0.001869,0.000671,0.000561,0.000781,0.000211,0.000822
3,4276,0.003005,0.002358,0.001752,0.002408,0.000443,0.002908,0.003749,0.001627,0.001471,...,0.007534,0.004774,0.006054,0.007393,0.009109,0.007312,0.004261,0.003791,0.000294,0.002026
4,4277,0.002374,0.002770,0.001659,0.002846,0.000396,0.003599,0.004504,0.001979,0.001979,...,0.009183,0.004862,0.008033,0.009664,0.012759,0.012378,0.006780,0.004429,0.000396,0.002054


In [11]:
dfso2.to_csv("./classif/10x10/classif10x10weekminute_smsout.csv")
dfso2.head()

,cellid,smsout0,smsout10,smsout20,smsout30,smsout40,smsout50,smsout60,smsout70,smsout80,...,smsout9980,smsout9990,smsout10000,smsout10010,smsout10020,smsout10030,smsout10040,smsout10050,smsout10060,smsout10070
0,4273,0.000756,0.000086,0.000980,0.000000,0.000288,0.000224,0.000000,0.000460,0.000804,...,0.001943,0.002133,0.002716,0.002752,0.000671,0.000671,0.000756,0.000671,0.001341,0.000309
1,4274,0.000156,0.000119,0.000063,0.000148,0.001456,0.000056,0.000000,0.001471,0.001483,...,0.000295,0.000566,0.000153,0.000618,0.000204,0.000130,0.000063,0.000019,0.000130,0.000026
2,4275,0.001508,0.000948,0.000457,0.001220,0.001692,0.000437,0.000134,0.001650,0.001662,...,0.002434,0.001820,0.001159,0.001935,0.001846,0.001535,0.000564,0.000285,0.001101,0.000192
3,4276,0.005051,0.001268,0.000274,0.003114,0.001221,0.000274,0.002632,0.000141,0.001130,...,0.008252,0.006963,0.001720,0.005474,0.006966,0.010124,0.005266,0.002900,0.001720,0.000451
4,4277,0.005813,0.000969,0.000000,0.002999,0.001453,0.000000,0.003437,0.000000,0.001453,...,0.009296,0.008281,0.001453,0.007768,0.010196,0.017054,0.008789,0.003483,0.001453,0.000484


In [12]:
dfci2.to_csv("./classif/10x10/classif10x10weekminute_callin.csv")
dfci2.head()

,cellid,callin0,callin10,callin20,callin30,callin40,callin50,callin60,callin70,callin80,...,callin9980,callin9990,callin10000,callin10010,callin10020,callin10030,callin10040,callin10050,callin10060,callin10070
0,4273,0.000998,0.000000,0.000693,0.000000,0.000000,0.000000,0.011474,0.000826,0.0,...,0.002713,0.004657,0.002020,0.001076,0.002112,0.001978,0.000593,0.000000,0.000000,0.000000
1,4274,0.004914,0.000050,0.001144,0.000099,0.000050,0.000099,0.008120,0.000070,0.0,...,0.001313,0.000388,0.000020,0.000190,0.002482,0.001205,0.001318,0.000050,0.000050,0.000050
2,4275,0.005405,0.000389,0.001222,0.001066,0.000533,0.000779,0.004394,0.000584,0.0,...,0.002370,0.003116,0.000339,0.002501,0.002646,0.001377,0.000645,0.001548,0.000533,0.000533
3,4276,0.001338,0.000244,0.005349,0.006555,0.002167,0.000488,0.000961,0.002167,0.0,...,0.007341,0.005554,0.005662,0.005571,0.002198,0.007570,0.000961,0.004121,0.004388,0.002167
4,4277,0.001294,0.000000,0.006591,0.007885,0.002587,0.000000,0.001294,0.002587,0.0,...,0.008009,0.005174,0.009071,0.005174,0.001294,0.009302,0.001294,0.003881,0.005298,0.002587


In [13]:
dfco2.to_csv("./classif/10x10/classif10x10weekminute_callout.csv")
dfco2.head()

,cellid,callout0,callout10,callout20,callout30,callout40,callout50,callout60,callout70,callout80,...,callout9980,callout9990,callout10000,callout10010,callout10020,callout10030,callout10040,callout10050,callout10060,callout10070
0,4273,0.003260,0.003121,0.002708,0.003451,0.001158,0.000000,0.003330,0.0,0.0,...,0.001830,0.012409,0.013541,0.004951,0.000206,0.004069,0.000949,0.000412,0.000622,0.000000
1,4274,0.002038,0.000332,0.001377,0.000358,0.002790,0.000045,0.001234,0.0,0.0,...,0.003020,0.001971,0.001255,0.005751,0.000063,0.001272,0.001074,0.001029,0.001028,0.000134
2,4275,0.002569,0.000921,0.003106,0.001160,0.002312,0.000350,0.000904,0.0,0.0,...,0.005915,0.004101,0.002118,0.006617,0.001372,0.001956,0.002149,0.001800,0.001098,0.001372
3,4276,0.001362,0.002166,0.005038,0.001521,0.001995,0.000219,0.000219,0.0,0.0,...,0.012617,0.006340,0.009093,0.003628,0.006829,0.003030,0.004017,0.002166,0.002810,0.004976
4,4277,0.001162,0.002324,0.004759,0.001162,0.002435,0.000000,0.000000,0.0,0.0,...,0.015343,0.005921,0.011746,0.003597,0.007194,0.003486,0.005825,0.002324,0.003486,0.005810


In [14]:
dfin2.to_csv("./classif/10x10/classif10x10weekminute_internet.csv")
dfin2.head()

,cellid,internet0,internet10,internet20,internet30,internet40,internet50,internet60,internet70,internet80,...,internet9980,internet9990,internet10000,internet10010,internet10020,internet10030,internet10040,internet10050,internet10060,internet10070
0,4273,0.020315,0.023687,0.046640,0.030528,0.065410,0.088188,0.058798,0.018274,0.019348,...,0.005482,0.004385,0.004216,0.004432,0.004716,0.005041,0.004862,0.004476,0.004206,0.005074
1,4274,0.014252,0.013623,0.011923,0.012707,0.013379,0.017332,0.012724,0.007810,0.009990,...,0.002345,0.001761,0.001605,0.002394,0.002333,0.002570,0.001698,0.002176,0.002127,0.003057
2,4275,0.017941,0.018990,0.012996,0.014538,0.012873,0.014881,0.011704,0.010763,0.012079,...,0.007045,0.006852,0.006848,0.007489,0.009070,0.007779,0.006668,0.006371,0.006397,0.007549
3,4276,0.031410,0.028836,0.028033,0.023313,0.021959,0.026398,0.026663,0.022209,0.021359,...,0.034443,0.042260,0.032566,0.036913,0.034449,0.033200,0.028067,0.026367,0.030207,0.037577
4,4277,0.034625,0.028813,0.031334,0.025597,0.023834,0.029168,0.030587,0.024091,0.023788,...,0.039142,0.048264,0.036600,0.041634,0.036265,0.036545,0.031216,0.029390,0.033604,0.042785
